## Basic demo notebook

First we import the libraries we need

In [ ]:
# Standard imports
import sys
import numpy as np
import matplotlib.pyplot as plt

# Third-party imports
import camb # Calculates the linear matter power spectrum

# Halo Model library imports
sys.path.append('../src')
import halomodel as halo

Now we set the cosmological parameters. If `sigma_8_set = True` we scale the linear power spectrum to account for the new `sigma_8` value.  

In [ ]:
# Set cosmological parameters
Omega_c = 0.25
Omega_b = 0.05
Omega_k = 0.0
h = 0.7
As = 2e-9
ns = 0.96
w = -1.0
wa = 0.0
m_nu = 0.0 # [eV]
sigma_8_set = True # If True uses the following value
sigma_8  = 0.8

# Wavenumber range [h/Mpc]
kmin, kmax = 1e-3, 10.
nk = 101
ks = np.logspace(np.log10(kmin), np.log10(kmax), nk)

# Redshift
z = 0.

# Halo mass range [Msun/h] over which to integrate
Mmin, Mmax = 1e9, 1e17
nM = 256
Ms = np.logspace(np.log10(Mmin), np.log10(Mmax), nM)

# CAMB
zmax_CAMB = 2.
kmax_CAMB = 200.

# Plot colours
col_lin = 'grey'
col_mat = 'C0'
col_gal = 'C1'
col_mg  = 'C4'

# Plot line styles
ls_li = '-'
ls_hm = '-'
ls_2h = '--'
ls_1h = ':'

# Plot labels
klab = r'$k\,/\,h \mathrm{Mpc}^{-1}$'
Pklab = r'$P(k)/(h^{-1}\,\mathrm{Mpc})^3$'

Now we initialise `CAMB` and produce a linear matter power spectrum. We have the option here to scale the power by an input `sigma_8` value.

In [ ]:
# Sets cosmological parameters in camb to calculate the linear power spectrum
pars = camb.CAMBparams()
wb = Omega_b*h**2
wc = Omega_c*h**2

# This function sets standard and helium set using BBN consistency
pars.set_cosmology(ombh2=wb, omch2=wc, H0=100.*h, mnu=m_nu, omk=Omega_k)
pars.set_dark_energy(w=w, wa=wa, dark_energy_model='ppf') 
pars.InitPower.set_params(As=As, ns=ns, r=0.)
pars.set_matter_power(redshifts=[z], kmax=kmax_CAMB) # Setup the linear matter power spectrum

# Scale 'As' to be correct for the desired 'sigma_8' value if necessary
if sigma_8_set:
    camb_results = camb.get_results(pars)
    sigma_8_init = (camb_results.get_sigma8()[[z].index(0.)]).item()
    print('Initial sigma_8:', sigma_8_init)
    scaling = (sigma_8/sigma_8_init)**2
    As *= scaling
    pars.InitPower.set_params(As=As, ns=ns, r=0.)

# Now get the linear power spectrum
Pk_lin = camb.get_matter_power_interpolator(pars, 
                                            nonlinear=False, 
                                            hubble_units=True, 
                                            k_hunit=True, 
                                            kmax=kmax_CAMB,
                                            var1=camb.model.Transfer_tot,
                                            var2=camb.model.Transfer_tot, 
                                            zmax=zmax_CAMB,
                                           )
Omega_m  = pars.omegam # Also extract the matter density
Pks_lin = Pk_lin.P(z, ks) # Single out the linear P(k) interpolator and evaluate linear power
camb_results = camb.get_results(pars)
sigma_8 = (camb_results.get_sigma8()[[z].index(0.)]).item()
if sigma_8_set: print('Final sigma_8:', sigma_8)

Now we can create a halo model, which needs to be reinitialised at each different redshift. In this example notebook we are only doing calculations at a single redshift, so we initialise the halo model with that in mind. We also need to choose a mass function and linear halo bias, in this example these come from `Tinker et al. (2010)`, and we need to choose a halo definition, here we choose $\Delta_\mathrm{v}=330$, so haloes are defined to be spherical objects that contain an average density that is $330$ times greater than the mean background universe, which is approximately the virial (spherical-collapse) definition for this cosmology at $z=0$.

In [ ]:
# Initialise halo model
hmod = halo.model(z, Omega_m, name='Tinker et al. (2010)', Dv=330.)
print(hmod)

Now we can find the Lagrangian radii, `Rs`, corresponding the the halo masses, then we can get an array of `sigma(R)` values. This requires an integral over the linear power spectrum times the Fourier transform of a top hat function, which is the oscillatory $\mathrm{sinc}(kr)$ function. We can get this from `CAMB`.

In [ ]:
# Get sigma(R) from CAMB
Rs = hmod.Lagrangian_radius(Ms)
sigmaRs = camb_results.get_sigmaR(Rs, hubble_units=True, return_R_z=False)[[z].index(z)]

First, we will use the halo model to compute the matter power spectrum, to do this we need to define a matter halo. We use the `profile` class, which computes the Fourier window function of the halo profile `W(M,k)` that is necessary for the power-spectrum calculation. We break the function $W(M,k)=A(M)U(M,k)/\bar{n}$ where $W(M,k)$ has dimensions of field multiplied by volume, $A(M)$, or `amp` is the amplitude the field value in each halo profile, and $\bar{n}$, or `norm` is the field normalisation. Here $A(M)/\bar{n}$ has dimension of field multiplied by volume and $U(M,k)$ is dimensionless and should tend to unity as k tends to zero, because its real space counter part is normalised.

In our case, we are going to compute the power spectrum of matter overdensity (density contrast: $\delta_{\rm m}=(\rho-\bar\rho)/\bar\rho$), which is dimensionless. We take $A(M)=M$ (`amp=Ms`), $\bar{n}=\bar\rho$ (`norm=hmod.rhom`), such that $N(M)/\bar{n}$ = $M/\bar\rho$ and has dimensions of volume. We could also have set $N(M)=M/\bar\rho$ and $\bar{n}=1$ and would get the same results. The distinction between these two approaches is important for discrete tracers.

We also need to set the optional `mass_tracer=True` in the halo profile, to let it know that our profile corresponds to a matter profile. This is important because the contribution to the matter power of haloes below `Mmin`, set above for halo-model integration, is important to the calculation (much of the mass in the Universe is in very low mass haloes), and this flag allows this to be taken into account in a consistent way.

In [ ]:
# Halo window functions
rvs = hmod.virial_radius(Ms)
cs = halo.concentration(Ms, z, method='Duffy et al. (2008)', halo_definition='Mvir')
Uk = halo.window_function(ks, rvs, cs, profile='NFW')

# Create a profile, need mass_tracer=True here
matter_profile = halo.profile.Fourier(ks, Ms, Uk, amp=Ms, norm=hmod.rhom, mass_tracer=True) 
print(matter_profile)

Now we are in a position to combine our halo model with our matter profile and do a calculation of the power spectrum. Note that profile must be provided as a dictionary: `{'m': matter_profile}` rather than `matter_profile`, where the key `'m'` can be chosen by the user and identifies that the `matter_profile` corresponds to field `'m'`. The reason for this will become apparent soon.

In [ ]:
# Calculate the halo-model power spectrum
Pk_2h, Pk_1h, Pk_hm = hmod.power_spectrum(ks, Pks_lin, Ms, sigmaRs, {'m': matter_profile})

Now let's do some plotting. In this first example we have only provided one profile, the `matter_profile`. So to access the power spectrum we need `Pk_xx['m-m']`.

In [ ]:
# Initialise plot
plt.subplots(3, 1, figsize=(6., 7.), dpi=100)
Pkmin, Pkmax = 1e1, 1e5
rmin, rmax = 1e-1, 1e2
smin, smax = 0.0, 1.1
kmin_plot, kmax_plot = 1e-3, 1e1

# P(k)
plt.subplot(3, 1, 1)
plt.loglog(ks, Pks_lin, color=col_lin, label='Linear')
plt.loglog(ks, Pk_2h['m-m'], color=col_mat, ls=ls_2h, label='Two-halo term')
plt.loglog(ks, Pk_1h['m-m'], color=col_mat, ls=ls_1h, label='One-halo term')
plt.loglog(ks, Pk_hm['m-m'], color=col_mat, ls=ls_hm, label='Matter')
plt.xticks([])
plt.xlim((kmin_plot, kmax_plot))
plt.ylabel(r'$P_\mathrm{mm}(k)\,/\,(h^{-1} \mathrm{Mpc})^3$')
plt.ylim((Pkmin, Pkmax))
plt.legend(fontsize='9', loc='lower left')

# Residual with linear
plt.subplot(3, 1, 2)
plt.loglog(ks, Pks_lin/Pks_lin, color=col_lin)
plt.loglog(ks, Pk_2h['m-m']/Pks_lin, color=col_mat, ls=ls_2h)
plt.loglog(ks, Pk_1h['m-m']/Pks_lin, color=col_mat, ls=ls_1h)
plt.loglog(ks, Pk_hm['m-m']/Pks_lin, color=col_mat, ls=ls_hm)
plt.xticks([])
plt.xlim((kmin_plot, kmax_plot))
plt.ylabel(r'$P_\mathrm{mm}(k)\,/\,P^\mathrm{lin}_{\rm mm}(k)$')
plt.ylim((rmin, rmax))

# Residual with halo-model matter power
plt.subplot(3, 1, 3)
plt.semilogx(ks, Pks_lin/Pk_hm['m-m'], color=col_lin, ls='-')
plt.semilogx(ks, Pk_2h['m-m']/Pk_hm['m-m'], color=col_mat, ls=ls_2h)
plt.semilogx(ks, Pk_1h['m-m']/Pk_hm['m-m'], color=col_mat, ls=ls_1h)
plt.semilogx(ks, Pk_hm['m-m']/Pk_hm['m-m'], color=col_mat, ls=ls_hm)
plt.xlabel(klab)
plt.xlim((kmin_plot, kmax_plot))
plt.ylabel(r'$P_\mathrm{mm}(k)\,/\,P_\mathrm{mm}^\mathrm{hm}(k)$')
plt.ylim((smin, smax))

# Finish
plt.tight_layout()
plt.show()

The upper panel shows the halo-model matter power spectrum broken into two- and one-halo components, we see that the two-halo term dominates at large scales while the one-halo term dominates at small scales, as expected. We see that the one-halo term has a constant 'shot-noise' contribution at large scales, and we see that the two-halo term deviates from the linear spectrum only at small scales, and only where the one-halo term is dominating the overall spectrum. The shot-noise contribution of the one-halo term at large scales is unphysical, and arises because neither mass nor momentum conservation have been imposed in the initial step of the derivation of the halo model, but this has very little impact in practice, although the one-halo term will eventually (and unphysically) dominate the power at ultra-large scales. The middle panel shows each spectra relative to linear and the bottom shows the contribution of the two- and one-halo terms to the total. Note that each term contributes equally at $k\sim 0.5 h\mathrm{Mpc}^{-1}$, which is the transition region at $z=0$. The slight (and unphysical) deviation of the total power from the linear result can be seen at $k\sim 10^{-3} h\mathrm{Mpc}^{-1}$.

Now let's calculate some galaxy spectra, first we need to specify a halo-occupation distribution (HOD) model to assign galaxies to haloes. In this example we choose a simple HOD, one that illustrates the basic point. Then we must calculate the mean galaxy density (which normalises the galaxy power spectrum) and finally create a galaxy `profile`. Here we use an isothermal profile for galaxies.

In [ ]:
# Simple HOD model
def HOD(M, Mmin):
    return np.rint(M/Mmin)
Ng = HOD(Ms, Mmin=1e11) # Number of galaxies at each halo mass

# Compute the mean galaxy density corresponding to our HOD
rhog = hmod.average(Ms, sigmaRs, Ng)
print('Mean galaxy density [(Mpc/h)^-3]:', rhog)

# Initialise profile class, we need discrete_tracer=True here because galaxies are discrete
Uk = halo.window_function(ks, rvs, profile='isothermal')
galaxy_profile = halo.profile.Fourier(ks, Ms, Uk, amp=Ng, var=Ng, norm=rhog, discrete_tracer=True) 
print(galaxy_profile)

Now that we have created both matter and galaxy profiles, it is easy to cross correlate them by supplying a list of the two halo profiles. When we do this, the code calculates both auto spectra and also the cross spectrum. If we supply `n` halo profiles then we get the triangle number of `n` independent auto/cross spectra computed. These are stored and accessed via the keys of `Pk_xx`. For example, if we have `profiles = {'u': profile_u, 'v': profile_v}` then `Pk_xx['u-u']` will store the auto spectrum corresponding to `profile_u`, `Pk_xx['v-v']` will store the same but for `profile_v`, the cross spectra can be accessed via either of `Pk_xx['u-v']` or  `Pk_xx['v-u']`, so there is some redundancy but the cross calculations are performed only once.

In [ ]:
# Calculate the halo-model power spectrum
profiles = {'m': matter_profile, 'g': galaxy_profile}
Pk_2h, Pk_1h, Pk_hm = hmod.power_spectrum(ks, Pks_lin, Ms, sigmaRs, profiles)

Now we can plot, noting that `Pk_xx['m-m']` singles out the matter-matter (auto) spectrum, `Pk_xx['g-g']` singles out the galaxy-galaxy (auto) spectrum, and `Pk_xx['m-g']` singles out the matter-galaxy cross spectrum, and is identical to `Pk_xx['g-m']`:

In [ ]:
# Initialise plot
plt.subplots(3, 1, figsize=(6., 7.))
Pkmin, Pkmax = 1e1, 1e5
kmin_plot, kmax_plot = 1e-3, 1e1
rmin, rmax = 1e-1, 3e2
smin, smax = 0., 5.5

# Lists for plots
Pks = [Pk_2h['m-m'], Pk_1h['m-m'], Pk_hm['m-m'], Pk_hm['g-g'], Pk_hm['m-g']]
cols = 3*[col_mat]+[col_gal, col_mg]
lss = [ls_2h, ls_1h, ls_hm, ls_hm, ls_hm]
labs = [None, None, 'Matter', 'Galaxy', 'Matter-galaxy']

# P(k)
plt.subplot(3, 1, 1)
plt.loglog(ks, Pks_lin, color=col_lin, label='Linear')
for (ls, lab) in zip([ls_2h, ls_1h], ['Two-halo term', 'One-halo term']):
    plt.plot(np.nan, ls=ls, label=lab, color='black')
for (Pk, col, ls, lab) in zip(Pks, cols, lss, labs):
    plt.loglog(ks, Pk, color=col, ls=ls, label=lab)
plt.xticks([])
plt.xlim((kmin_plot, kmax_plot))
plt.ylabel(r'$P_{\rm uv}(k)\,/\,(h^{-1} \mathrm{Mpc})^3$')
plt.ylim((Pkmin, Pkmax))
plt.legend(ncol=2, loc='lower left', fontsize='9')

# Lists for plots
Pks = [Pk_2h['m-m'], Pk_1h['m-m'], Pk_hm['m-m'],
       Pk_2h['g-g'], Pk_1h['g-g'], Pk_hm['g-g'],
       Pk_2h['m-g'], Pk_1h['m-g'], Pk_hm['m-g']]
cols = 3*[col_mat]+3*[col_gal]+3*[col_mg]
lss = 3*[ls_2h, ls_1h, ls_hm]

# Residual with linear
plt.subplot(3, 1, 2)
plt.loglog(ks, Pks_lin/Pks_lin, color=col_lin)
for (Pk, col, ls) in zip(Pks, cols, lss):
    plt.loglog(ks, Pk/Pks_lin, color=col, ls=ls)
plt.xticks([])
plt.xlim((kmin_plot, kmax_plot))
plt.ylabel(r'$P_{\rm uv}(k)\,/\,P^\mathrm{lin}(k)$')
plt.ylim((rmin, rmax))

# Residual with halo-model matter power
plt.subplot(3, 1, 3)
plt.semilogx(ks, Pks_lin/Pk_hm['m-m'], color=col_lin)
for (Pk, col, ls) in zip(Pks, cols, lss):
    plt.semilogx(ks, Pk/Pk_hm['m-m'], color=col, ls=ls)
plt.xlabel(klab)
plt.xlim((kmin_plot, kmax_plot))
plt.ylabel(r'$P_{\rm uv}(k)\,/\,P_\mathrm{mm}(k)$')
plt.ylim((smin, smax))

# Finish
plt.show()

We see that all spectra have the same shape at large scales, but that those spectra that involve galaxies are offset in amplitude. This tells us that the galaxy sample is positively biased, with $b\simeq1.5$. At smaller scales the spectra are all similarly shaped, but the exact scale dependence is different for each. This arises because of the different way that matter and galaxies occupy haloes.

We can also use `halomodel` to calculate the 3D spectra relevant to tSZ, for this we use the 'universal pressure profile' (Arnaud et al. 2010) to for the halo electron pressure distribution. In the example here we show how to create `profile`s in configuration (real) space, as opposed to Fourier space. Using configuration-space profiles is convenient for many applications of the halo model, but comes with some additional overhead as the profiles need to be Fourier transformed internally, which is (relatively) computationally expensive.

In [ ]:
# Universal pressure profile: UPP
def rho_UPP(r, M, rv, c, z, Om_m):
    alphap = 0.12
    r500 = rv/2. # Good enough approximation to this example
    H0 = 100. # TODO: Not sure on units here!
    def p(x):
        P0, c500, alpha, beta, gamma = 6.41, 1.81, 1.33, 4.13, 0.31
        y = c500*x
        f1 = y**(2.-gamma)
        f2 = (1.+y**alpha)**(beta-gamma)/alpha
        return P0*(h/0.7)**(-3./2.)*f1*(r500/c500)**2/f2
    a = 1./(1.+z)
    H = H0*np.sqrt(Om_m*(1.+z)**3+(1.-Om_m)) # Hubble parameter assuming late-time LCDM
    f1 = 1.65*(h/0.7)**2*H**(8./3.)
    f2 = (M/2.1e14)**(2./3.+alphap)
    return f1*f2*p(r/r500)*4.*np.pi
Prho_UPP = lambda r, M, rv, c: rho_UPP(r, M, rv, c, z, hmod.Om_m)

# NFW profile (multiplied by 4pir^2 with constant factors removed)
def Prho_NFW(r, M, rv, c):
    rs = rv/c
    return r/(1.+r/rs)**2

# Isothermal profile (multiplied by 4pir^2 with constant factors removed gives a constant)
def Prho_iso(r, M, rv, c):
    return 1.

# Create profiles in configuration space
print('Matter profile')
matter_profile = halo.profile.configuration(ks, Ms, Prho_NFW, rvs, cs, amp=Ms, norm=hmod.rhom, mass_tracer=True)
print(matter_profile)
print('Galaxy profile')
galaxy_profile = halo.profile.configuration(ks, Ms, Prho_iso, rvs, cs, amp=Ng, norm=rhog, discrete_tracer=True)
print(galaxy_profile)
print('Pressure profile')
pressure_profile = halo.profile.configuration(ks, Ms, Prho_UPP, rvs, cs)
print(pressure_profile)

# Power spectrum calculation
halo_profiles = {'matter': matter_profile, 'galaxy': galaxy_profile, 'pressure': pressure_profile}
Pk_2h, Pk_1h, Pk_hm = hmod.power_spectrum(ks, Pks_lin, Ms, sigmaRs, halo_profiles)

In [ ]:
# Initialise
plt.subplots(1, 2, figsize=(10, 4))
Pkmin, Pkmax = 1e1, 1e5
pfac = 0.02 # Scaling factor for pressure to bring it into the same range as other spectra

# Auto spectra
plt.subplot(1, 2, 1)
for name, col, fac in zip(['matter-matter', 'galaxy-galaxy', 'pressure-pressure'], [col_mat, col_gal, 'purple'], [1., 1., pfac**2]):
    plt.loglog(ks, fac*Pk_2h[name], ls=ls_2h, color=col)
    plt.loglog(ks, fac*Pk_1h[name], ls=ls_1h, color=col)
    plt.loglog(ks, fac*Pk_hm[name], ls=ls_hm, color=col, label=name)
plt.xlabel(klab)
plt.xlim((kmin, kmax))
plt.ylabel(Pklab)
plt.ylim((Pkmin, Pkmax))
plt.legend()

# Cross spectra
plt.subplot(1, 2, 2)
for name, col, fac in zip(['matter-galaxy', 'matter-pressure', 'galaxy-pressure'], ['royalblue', 'forestgreen', 'deeppink'], [1., pfac, pfac]):
    plt.loglog(ks, fac*Pk_2h[name], ls=ls_2h, color=col)
    plt.loglog(ks, fac*Pk_1h[name], ls=ls_1h, color=col)
    plt.loglog(ks, fac*Pk_hm[name], ls=ls_hm, color=col, label=name)
plt.xlabel(klab)
plt.xlim((kmin, kmax))
plt.yticks([])
plt.ylim((Pkmin, Pkmax))
plt.legend()

# Finalise
plt.tight_layout()
plt.show()

We see that we can compute the auto and cross spectra of a new tracer very easily using `halomodel`.